In [7]:
import requests
from bs4 import BeautifulSoup
import re
import csv
# URL of the Wikihow page to scrape
url = 'https://www.wikihow.com/Special: Randomizer'
 # Send an HTTP request to the URL and receive the HTML content
response = requests.get(url)
html_content = response.content
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')
article_title = soup.find('title').text.strip()
print(article_title)


Log in to view


In [8]:
article_title = soup.find('title').text.strip()
print(article_title)

Log in to view


In [9]:
#Extract the subheadings and paragraphs using the appropriate HTML tags
subheadings = []
paragraphs = []
steps = soup.find_all('div', {'class': 'step'})
for step in steps:
    subheading_element = step.find('b')
    if (subheading_element is not None):
        subheading_text = subheading_element.text.strip().replace('\n','')
        subheading_text = subheading_text.encode('ascii', errors="ignore").decode()
        subheading_text = re.sub(r",", subheading_text)
        print(subheading_text)
        subheadings.append(subheading_text)
        
        #this block removes titles and extra links
        subheading_element.extract()
        for span_tag in step.find_all('span'):
            span_tag.extract()
        paragraph_text = step.text.strip().replace('\n','').replace('','')
        paragraph_text = paragraph_text.encode("ascii", errors='ignore').decode()
        paragraph_text = re.sub(",", paragraph_text) 
        print(paragraph_text)
        paragraphs.append(paragraph_text)

if (len(subheadings)):
    with open("/kaggle/working/wikiHow.csv", mode='a', newline=",", encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file) 
    for i in range(len(subheadings)):
        writer.writerow([article_title, subheadings[i], paragraphs[i]])


In [10]:
max_input_length = 1024
max_output_length = 64
batch_size = 16
def process_data_to_model_inputs(batch):
#tokenize the inputs and labels
    inputs = tokenizer(
        batch["paragraph"],
        padding="max length", 
        truncation=True,
        max_length=max_input_length,
	)
    outputs = tokenizer(
	    batch["heading"],
        padding="max length",
        truncation=True,
        max_length=max_output_length,
	)
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    
    #create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
    [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    #since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids
    #We have to make sure that the PAD token is ignored
    batch["labels"] = [ 
    [-100 if token == tokenizer.pad_token_id else token for token in labels]
    for labels in batch["labels"]
    ]
    return batch
